In [ ]:
import pprint 
import client
import middleware

pp = pprint.PrettyPrinter(indent=2)

In [ ]:
data = client.parse_client_script('../logs/maximum-throughput/thread-10-client-100/repetition-1-client-1.log')
pp.pprint(data)

In [ ]:
import csv
import operator
import os

import matplotlib

import matplotlib.pyplot as plt
import numpy as np

# Enable inline plotting
%matplotlib inline
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica Neue') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 11})

In [ ]:
base_dir = '/Users/ruifengxu/Development/asl-fall16-project/log/'
trace_filename = os.path.join(base_dir, 'trace.log');

# Thread id (0)
# Log level (1)
# Completion time (2)
# Command (3)
# Transaction id (4)
# Server id (5)
# Success flag (6)
# Cache miss flag (7)
# Time to read request (8)
# Time in queue (9)
# Time to forward request (10)
# Time to receive response (11)
# Time to send response (12)
#
# Example
# 33,I,1477020519213,g,78137700,2,s,h,2900,6703800,8500,1904400,6100

In [ ]:
middleware.exam_logs(trace_filename, level='D', output=True)

In [ ]:
middleware.exam_logs(trace_filename, level='E', output=True)

In [ ]:
traces = middleware.load_trace(trace_filename)

In [ ]:
def server_load_distribution(traces):
  dist = {}  
  for row in traces:
    if row[5] not in dist:
      dist[row[5]] = 1
    else:
      dist[row[5]] += 1
  return dist

In [ ]:
load_dist = server_load_distribution(traces)
print(load_dist)

In [ ]:
def plot_load_dist(load_dist):
  fig = plt.figure( figsize=(6,4))
  plt.bar(range(len(load_dist)), 
          load_dist.values(), 
          align='center',
          alpha=0.5,
          color='blue')
  plt.xticks(range(len(load_dist)), load_dist.keys())
  plt.xlabel("Server ID")
  plt.ylabel("Number of Requests")
  plt.tight_layout()
  plt.show()
  fig.savefig('load-dist.png', dpi=300)

In [ ]:
plot_load_dist(load_dist)

In [ ]:
def calculate_throughput(traces, interval=1, sample_frequency=100):
  throughput = []
  current_time = None
  next_time = None 
  count = 0
  for row in traces:
    if not row[6]:
      continue
      
    if current_time is None:
      current_time = row[2]
      next_time = row[2] + interval
    
    if row[2] < next_time:
      count += 1
    else:
      throughput.append(count * sample_frequency)
      next_time += interval
      count = 1

  return throughput

In [ ]:
throughput = calculate_throughput(traces, interval=1, sample_frequency=100)

In [ ]:
def plot_throughput(throughput, start, end):
  fig = plt.figure( figsize=(16,6))
  x = np.arange(0, end - start, 1)
  lines = plt.plot(x, throughput[start: end], 'blue')
  for line in lines:
    line.set_alpha(0.6)
    line.set_linewidth(0.5)
  plt.ylabel('Throughput (requests/s)')
  plt.xlabel('Time (s)')
  plt.xlim([0, 60])
  plt.tight_layout()
  plt.show()  
  fig.savefig('throughput.png', dpi=300)

In [ ]:
plot_throughput(throughput, 0, 60)

In [ ]:
def calculate_response_time(traces, interval=1):
  response_time_avg = []
  response_time_std = []
  current_time = None
  next_time = None 
  entries = []
  for row in traces:
    if not row[2]:
      continue
      
    if current_time is None:
      current_time = row[2]
      next_time = row[2] + interval
    
    time = (row[8] + row[9] + row[10] + row[11] + row[12]) / 1000000
    
    if row[2] < next_time:      
      entries.append(time)
    else:
      response_time_avg.append(np.mean(entries))
      response_time_std.append(np.std(entries))
      next_time += interval
      entries = [time]

  return response_time_avg, response_time_std

In [ ]:
response_time_avg, response_time_std = calculate_response_time(traces, interval=1)

In [ ]:
def plot_response_time(response_time_avg, response_time_std, start, end):
  fig = plt.figure( figsize=(16,6))
  x = np.arange(0, end - start, 1)
  y = response_time_avg[start: end]
  std = response_time_std[start: end]
  lines = plt.plot(x, y, 'blue')
  for line in lines:
    line.set_alpha(0.6)
    line.set_linewidth(0.5)
  plt.fill_between(x, np.subtract(y, std), np.add(y, std), 
                     alpha=0.20, edgecolor='#FFFFFF', facecolor='blue')
  plt.ylabel('Response time (ms)')
  plt.xlabel('Time (s)')
  plt.xlim([0, 60])
  plt.ylim([0, 10])
  plt.tight_layout()
  plt.show()  
  fig.savefig('response-time.png', dpi=300)

In [ ]:
plot_response_time(response_time_avg, response_time_std, 0, 60)

In [ ]:
c1_1 = [(93867, 318, 233.64, 3116), (93684, 319, 228.91, 3110), (80422, 373, 294.81, 2659), (91435, 328, 235.47, 3023), (85110, 351, 257.81, 2825)]
c1_2 = [(86869, 344, 276.42, 2882), (79612, 375, 307.38, 2643), (81065, 370, 324.32, 2681), (85187, 351, 281.39, 2828), (83873, 356, 288.30, 2785)]

c10_1 = [(309200, 970, 1728.85, 10267), (308044, 975, 1739.90, 10216), (327009, 922, 1606.57, 10815), (327571, 915, 1625.00, 10875), (306296, 979, 1700.85, 10169)]
c10_2 = [(326268, 924, 1627.50, 10791), (329648, 914, 1625.37, 10907), (300725, 999, 1693.42, 9984), (306617, 983, 1614.64, 10131), (324459, 925, 1523.00, 10773)]

c20_1 = [(423852, 1424, 2282.24, 14012), (421330, 1427, 2408.44, 13987), (426850, 1414, 2258.86, 14109), (426299, 1409, 2451.93, 14154), (404643, 1485, 4859.02, 13433)]
c20_2 = [(407422, 1481, 2268.31, 13468), (403971, 1493, 2395.83, 13365), (406502, 1478, 2246.70, 13495), (413211, 1454, 2367.90, 13718), (413473, 1454, 4035.05, 13727)]

c30_1 = [(471198, 1913, 3040.33, 15644), (434542, 2076, 4973.88, 14426), (468532, 1925, 2999.80, 15554), (463903, 1952, 2889.32, 15326), (456173, 1979, 3106.01, 15129)]
c30_2 = [(428842, 2102, 3129.12, 14238), (470615, 1919, 4841.86, 15604), (449738, 2005, 3027.51, 14933), (436280, 2070, 2911.47,14464), (452325, 1997, 3057.47, 14995)]

c40_1 = [(506694, 2375, 3224.26, 16819),(458537, 2624, 5361.23, 15222), (506187, 2376, 3025.83, 16806), (473853, 2538, 3155.20, 15733), (487636, 2468, 7195.72, 16191)]
c40_2 = [(471398, 2551, 3257.70 ,15651), (491675, 2446, 5169.08, 16324), (459069, 2620, 3093.55, 15241), (505010, 2382, 3000.09, 16765), (482272, 2497, 7180.68, 15989)]

c50_1 = [(493988, 3047, 6537.80, 16396), (494317, 3041, 4935.96, 16411), (494445, 3041, 10618.62, 16412), (502937, 3005, 5516.11, 16624), (492702, 3052, 4570.71, 16358)]
c50_2 = [(485838, 3098, 6675.73, 16117), (506071, 2981, 4825.08, 16740), (463899, 3240, 10686.13, 15399), (490001, 3069, 5604.92, 16267), (516135, 2921, 3585.32, 17094)]

In [ ]:
def baseline(a, b):
  tp = 0
  rt = 0
  max_std = 0
  for i in range(0, 5):
    tp += a[i][3] 
    tp += b[i][3] 
    rt += a[i][1]
    rt += b[i][1]
    if max_std < a[i][2]:
      max_std = a[i][2]
    if max_std < b[i][2]:
      max_std = b[i][2]
  
  return tp / 5, rt / 10000, max_std / 1000

In [ ]:
tp = [
  baseline(c1_1, c1_2)[0], 
  baseline(c10_1, c10_2)[0], 
  baseline(c20_1, c20_2)[0], 
  baseline(c30_1, c30_2)[0],
  baseline(c40_1, c40_2)[0],
  baseline(c50_1, c50_2)[0],
]
rt = [
  baseline(c1_1, c1_2)[1], 
  baseline(c10_1, c10_2)[1], 
  baseline(c20_1, c20_2)[1], 
  baseline(c30_1, c30_2)[1],
  baseline(c40_1, c40_2)[1],
  baseline(c50_1, c50_2)[1],
]
std = [
  baseline(c1_1, c1_2)[2], 
  baseline(c10_1, c10_2)[2], 
  baseline(c20_1, c20_2)[2], 
  baseline(c30_1, c30_2)[2],
  baseline(c40_1, c40_2)[2],
  baseline(c50_1, c50_2)[2],
]

In [ ]:
fig = plt.figure(figsize=(16,6))
x = np.arange(0, 110, 20)
lines = plt.plot(x, tp, 'blue')
for line in lines:
    line.set_alpha(0.6)
    line.set_linewidth(0.5)
plt.ylabel('Aggregated throughput (requests/s)')
plt.xlabel('Number of clients')
plt.tight_layout()
plt.xlim([1, 110])
fig.savefig('baseline-throughput.png', dpi=300)

In [ ]:
fig = plt.figure(figsize=(16,6))
x = np.arange(0, 110, 20)
plt.errorbar(x, rt, yerr=std)
plt.ylabel('Average response time (ms)')
plt.xlabel('Number of clients')
plt.tight_layout()
plt.xlim([1, 110])
plt.ylim([0, 15])
plt.show()
fig.savefig('baseline-response-time.png', dpi=300)